<a href="https://colab.research.google.com/github/hanadoolsae/ewhaguidebook/blob/main/%08data/embedding/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install konlpy
!pip install transformers
!pip install torch
!pip install scikit-learn

In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
import pickle
import numpy as np  # Add this line
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 파일에서 불용어 불러오기
with open('/content/stopwords.txt', 'r', encoding='utf-8') as f:
    file_stopwords = f.readlines()
file_stopwords = [x.strip() for x in file_stopwords]

# 불용어 리스트
custom_stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '것', '라고', '에게', '라면', '을', '이라',
             '라니', '있다', '아', '랑', '쯤된', '에서', '에선', '어', '이지만', '으로나', '때', '때는', '때라면', '때라서', '라', '이다', '있', '죠', '고', '니', '로', '있',
             '같', '어서', '어요', '는데', '습니다', '면서', '많이', '마', '더', '그렇다', '의', '당', '좀', '책', '안', '볼', '게', '안', '정말', '듯', '이제야', '여', '요',
             '게다가', '같다', '임', '로서', '이제', '만', '인', '붙이', '그', '저', '수', '가제', '부터', '닷', '저희', '적', '알', '쉬', '못', '꼭', '살', '제', '권', '제',
             '분', '나', '내', '진작', '전', '뿐', '대한', '대해', '책', '좋다', '이야기','하는','통해','저자','독자','작가','위','위해','가장','중','더욱','실제','작품',
             '가지','지금','수록','설명','내용','모든','스스로','시리즈','부','때문','그것','소개','명','속','데','이후','판','주제']

# 두 불용어 리스트 합치기
stopwords = file_stopwords + custom_stopwords

# 한국어 텍스트 토큰화 및 불용어 제거 함수
tokenizer = Okt()

def tokenize_korean_text(text):
    text = re.sub('[^가-힣a-zA-Z0-9\s]', '', text)  # 특수문자 제거
    morphs_tokens = tokenizer.morphs(text)  # 모든 형태소 분석 # 형태소와 명사 토큰을 합칩니다.
    return ' '.join([token for token in morphs_tokens if token not in stopwords])

# 엑셀 파일 읽기
data = pd.read_csv('/content/book_data.csv')

# 'book_introduce' 열을 전처리하고 원본 데이터프레임에 저장
for idx in range(len(data)):
    if pd.notnull(data.loc[idx, 'book_introduce']):
        # NaN이 아닌 경우에만 전처리 수행
        preprocessed_text = tokenize_korean_text(data.loc[idx, 'book_introduce'])
        data.at[idx, 'book_introduce'] = preprocessed_text  # 전처리된 텍스트를 원본 데이터프레임에 저장

    # 1000개마다 로그 찍기
    if (idx+1) % 1000 == 0:
        print(f"Processed {idx+1} rows.")

# 전처리된 데이터를 피클 파일로 저장
with open('preprocessed_book_introduce.pkl', 'wb') as file:
    pickle.dump(data, file)  # 전체 데이터프레임을 저장

In [ ]:
# Klue-BERT 모델 및 토크나이저 불러오기
model = AutoModel.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# 텍스트 임베딩 함수
def embed_text(text):
    # 원본 텍스트를 토큰 ID로 인코딩
    input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=512)

    # 텐서로 변환
    input_ids = torch.tensor([input_ids])

    # 모델에 입력하여 마지막 은닉 상태를 얻음
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]

    # [CLS] 토큰의 임베딩을 반환
    return last_hidden_states[0, 0, :].numpy()

# 각 도서 소개를 임베딩하고 ISBN과 함께 저장
embedded_books_with_isbn = []

for isbn, tokens in book_train_X:
    # 토큰화된 명사들을 공백으로 구분된 문자열로 만듬
    nouns_text = ' '.join(tokens)
    # 해당 텍스트를 임베딩
    embedded_text = embed_text(nouns_text)
    # 임베딩과 ISBN을 튜플로 묶어 리스트에 추가
    embedded_books_with_isbn.append((isbn, embedded_text))

# 결과 확인
print(embedded_books_with_isbn[:5])  # 처음 5개의 결과만 출력


In [ ]:
# 도서 임베딩 벡터만 추출
embedded_books = np.array([emb for isbn, emb in embedded_books_with_isbn])

# 도서 간의 유사도 계산 함수
def calculate_similarity(embedded_books):
    # 모든 임베딩에 대한 코사인 유사도 계산
    similarity_matrix = cosine_similarity(embedded_books)
    return similarity_matrix

# 도서 간 유사도 계산
similarity_matrix = calculate_similarity(embedded_books)

# 유사도 행렬 출력 (예시로 처음 5x5개의 유사도만 출력)
print(similarity_matrix[:5, :5])


In [ ]:
# 임베딩된 벡터만을 추출합니다.
embedded_books = np.array([emb for _, emb in embedded_books_with_isbn])
# 만약 ISBN도 함께 저장하고 싶다면 별도의 파일로 저장할 수 있습니다.
isbns = [isbn for isbn, _ in embedded_books_with_isbn]
np.save('embedding_matrix.npy', isbns)
